In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

title - Предмет  
type - Вид  
date - Дата  
weekday - День недели
time - Время  
map - Место  
lecturer - Преподаватель

In [2]:
start = 5
end = 18
data = pd.DataFrame(columns = ['date', 'weekday', 'time_start', 'time_end', 'title', 'type', 'map', 'lecturer'])
j = 0
for i in range(start, end):
    response = requests.get('https://mai.ru/education/schedule/detail.php?group=%D0%9C3%D0%97-301%D0%91%D0%BA-17&week='+str(i))
    soup = BeautifulSoup(response.text)
    datetime = soup.findAll(class_ = 'sc-table-row')
    loc = soup.findAll(class_ = 'sc-table-col sc-item-location')
    k = 0
    for n, row in enumerate(datetime):
        if row.find(class_ = 'sc-table-col sc-day-header sc-gray'):
            date = row.find(class_ = 'sc-table-col sc-day-header sc-gray').text
            k += 1
            continue
        time = row.find(class_ = 'sc-table-col sc-item-time').text
        title = row.find(class_ = 'sc-title').text
        item_type = row.find(class_ = 'sc-table-col sc-item-type').text
        location = loc[n - k].text
        lecturer = row.find(class_ = 'sc-lecturer').text
        data.loc[j, 'date'] = date[:5]
        data.loc[j, 'weekday'] = date[-2:]
        data.loc[j, 'time_start'] = time[:5]
        data.loc[j, 'time_end'] = time[-5:]
        data.loc[j, 'title'] = title
        data.loc[j, 'type'] = item_type
        data.loc[j, 'map'] = location[1:]
        data.loc[j, 'lecturer'] = lecturer
        j += 1

In [3]:
data['date'] = pd.to_datetime(data['date']+'.2019', format = '%d-%m-%Y', errors='ignore')

In [4]:
data

,date,weekday,time_start,time_end,title,type,map,lecturer
0,01.10.2019,Вт,18:15,19:45,Информационные системы и технологии,ПЗ,ГУК Б-330Б,Смирнов Владимир Юрьевич
1,01.10.2019,Вт,20:00,21:30,Информационные системы и технологии,ПЗ,ГУК Б-330Б,Смирнов Владимир Юрьевич
2,02.10.2019,Ср,18:15,19:45,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
3,02.10.2019,Ср,20:00,21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
4,07.10.2019,Пн,18:15,19:45,Базы данных,ПЗ,3-422,Склеймин Юрий Борисович
...,...,...,...,...,...,...,...,...
57,19.12.2019,Чт,20:00,21:30,Теоретические основы информатики,ЛР,24-705,Ермаков Александр Александрович
58,21.12.2019,Сб,09:00,10:30,Иностранный язык,ПЗ,2-319(2А),Жаринова Евгения Геннадьевна
59,21.12.2019,Сб,10:45,12:15,Иностранный язык,ПЗ,2-319(2А),Жаринова Евгения Геннадьевна
60,24.12.2019,Вт,18:15,19:45,Практикум по информационным технологиям,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич


In [5]:
def dup(i):
    if (data.loc[i, 'date'] == data.loc[i+1, 'date'] and
    data.loc[i, 'title'] == data.loc[i+1, 'title'] and
    data.loc[i, 'map'] == data.loc[i+1, 'map'] and
    data.loc[i, 'lecturer'] == data.loc[i+1, 'lecturer']):
        data.loc[i, 'time_end'] = data.loc[i+1, 'time_end']

for i in range(data.shape[0] - 1):
    dup(i)

In [6]:
def dup2(i):
    if (data.loc[i, 'date'] == data.loc[i+2, 'date'] and
    data.loc[i, 'title'] == data.loc[i+2, 'title'] and
    data.loc[i, 'map'] == data.loc[i+2, 'map'] and
    data.loc[i, 'lecturer'] == data.loc[i+2, 'lecturer']):
        data.loc[i, 'time_end'] = data.loc[i+2, 'time_end']

for i in range(data.shape[0] - 2):
    dup2(i)

In [7]:
data = data.drop_duplicates(['date', 'time_end', 'title', 'map', 'lecturer'], keep = 'first')
data = data.reset_index(drop = True)
data

,date,weekday,time_start,time_end,title,type,map,lecturer
0,01.10.2019,Вт,18:15,21:30,Информационные системы и технологии,ПЗ,ГУК Б-330Б,Смирнов Владимир Юрьевич
1,02.10.2019,Ср,18:15,21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
2,07.10.2019,Пн,18:15,21:30,Базы данных,ПЗ,3-422,Склеймин Юрий Борисович
3,09.10.2019,Ср,18:15,21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
4,10.10.2019,Чт,18:15,21:30,Программная инженерия,ЛК,3-444,Гагарин Андрей Петрович
5,14.10.2019,Пн,18:15,21:30,Базы данных,ЛР,3-422,Склеймин Юрий Борисович
6,16.10.2019,Ср,18:15,21:30,Теоретические основы информатики,ЛР,24-705,Ермаков Александр Александрович
7,21.10.2019,Пн,18:15,21:30,Базы данных,ЛР,3-422,Склеймин Юрий Борисович
8,23.10.2019,Ср,18:15,21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
9,24.10.2019,Чт,18:15,21:30,Программная инженерия,ЛР,3-444,Гагарин Андрей Петрович


In [8]:
def time_period(i):
    data.loc[i, 'time'] = data.loc[i, 'time_start'] + ' - ' + data.loc[i, 'time_end']

for i in range(data.shape[0]):
    time_period(i)

In [9]:
del data['time_start']
del data['time_end']

In [10]:
exile = data['time']
data.drop(labels = ['time'], axis = 1, inplace = True)
data.insert(2, 'time', exile)
data.to_csv('rasp302.csv', index = False, encoding = 'utf-8')
data

,date,weekday,time,title,type,map,lecturer
0,01.10.2019,Вт,18:15 - 21:30,Информационные системы и технологии,ПЗ,ГУК Б-330Б,Смирнов Владимир Юрьевич
1,02.10.2019,Ср,18:15 - 21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
2,07.10.2019,Пн,18:15 - 21:30,Базы данных,ПЗ,3-422,Склеймин Юрий Борисович
3,09.10.2019,Ср,18:15 - 21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
4,10.10.2019,Чт,18:15 - 21:30,Программная инженерия,ЛК,3-444,Гагарин Андрей Петрович
5,14.10.2019,Пн,18:15 - 21:30,Базы данных,ЛР,3-422,Склеймин Юрий Борисович
6,16.10.2019,Ср,18:15 - 21:30,Теоретические основы информатики,ЛР,24-705,Ермаков Александр Александрович
7,21.10.2019,Пн,18:15 - 21:30,Базы данных,ЛР,3-422,Склеймин Юрий Борисович
8,23.10.2019,Ср,18:15 - 21:30,Информационные системы и технологии,ЛР,ГУК Б-330Б,Смирнов Владимир Юрьевич
9,24.10.2019,Чт,18:15 - 21:30,Программная инженерия,ЛР,3-444,Гагарин Андрей Петрович
